In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.min_rows', 500)

In [13]:
# Reading data from csv file
df_coffee = pd.read_csv("data/psd_coffee.csv")
df_coffee.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,711100,"Coffee, Green",AG,Algeria,2003,2012,6,29,Arabica Production,2,(1000 60 KG BAGS),0.0
1,711100,"Coffee, Green",AG,Algeria,2003,2012,6,90,Bean Exports,2,(1000 60 KG BAGS),0.0
2,711100,"Coffee, Green",AG,Algeria,2003,2012,6,58,Bean Imports,2,(1000 60 KG BAGS),1800.0
3,711100,"Coffee, Green",AG,Algeria,2003,2012,6,20,Beginning Stocks,2,(1000 60 KG BAGS),0.0
4,711100,"Coffee, Green",AG,Algeria,2003,2012,6,125,Domestic Consumption,2,(1000 60 KG BAGS),1805.0


In [14]:
# Removing unnecessary data(columns and rows)
df_coffee = df_coffee[['Country_Name', 'Market_Year',
                       'Attribute_Description','Value']]
attr_rmlist = ['Beginning Stocks', 'Ending Stocks', 'Other Production', 'Rst,Ground Dom. Consum', 'Soluble Dom. Cons.', 'Total Distribution', 'Total Supply']
df_coffee = df_coffee[~df_coffee['Attribute_Description'].isin(attr_rmlist)]
df_coffee.head(10)

,Country_Name,Market_Year,Attribute_Description,Value
0,Algeria,2003,Arabica Production,0.0
1,Algeria,2003,Bean Exports,0.0
2,Algeria,2003,Bean Imports,1800.0
4,Algeria,2003,Domestic Consumption,1805.0
6,Algeria,2003,Exports,0.0
7,Algeria,2003,Imports,1805.0
9,Algeria,2003,Production,0.0
10,Algeria,2003,Roast & Ground Exports,0.0
11,Algeria,2003,Roast & Ground Imports,0.0
12,Algeria,2003,Robusta Production,0.0


In [15]:
# Getting Country Code format, ISO Alpha-3 which other data sources follow
# url to scrape for ISO 3166 country codes Alpha-2 and Alpha-3 from www.iban.com
country_code_url ="https://www.iban.com/country-codes"
# Use panda's `read_html` to parse the url
df_countryCode = pd.read_html(country_code_url, header=0)[0]
# eliminating unnessasary data
df_countryCode = df_countryCode.iloc[:,[0,2]]
# rename the columns
df_countryCode.rename(columns={'Country':'Country_Name',
                               'Alpha-3 code':'Country_Code'
                              },inplace=True)
df_countryCode.head()

,Country_Name,Country_Code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [16]:
# Checking for Country names before merging
# List out Countries in df_coffee and not in df_countryCode
df_coffee[~df_coffee.Country_Name.isin(df_countryCode.Country_Name)].Country_Name.unique()

array(['Bolivia', 'Central African Republic', 'Congo (Brazzaville)',
       'Congo (Kinshasa)', "Cote d'Ivoire", 'Dominican Republic',
       'European Union', 'Iran', 'Korea, South', 'Laos', 'Philippines',
       'Russia', 'Taiwan', 'Tanzania', 'United States', 'Venezuela',
       'Vietnam', 'Yemen (Sanaa)'], dtype=object)

In [17]:
# Replacing names in the countryCode dataframe to match with df_coffee
df_countryCode.loc[(df_countryCode.Country_Name == 'Bolivia (Plurinational State of)'),'Country_Name']='Bolivia'
df_countryCode.loc[(df_countryCode.Country_Name == 'Central African Republic (the)'),'Country_Name']='Central African Republic'
df_countryCode.loc[(df_countryCode.Country_Name == 'Congo (the)'),'Country_Name']='Congo (Brazzaville)'
df_countryCode.loc[(df_countryCode.Country_Name == 'Congo (the Democratic Republic of the)'),'Country_Name']='Congo (Kinshasa)'
df_countryCode.loc[(df_countryCode.Country_Name == "Côte d'Ivoire"),'Country_Name']="Cote d'Ivoire"
df_countryCode.loc[(df_countryCode.Country_Name == 'Dominican Republic (the)'),'Country_Name']='Dominican Republic'
df_countryCode.loc[(df_countryCode.Country_Name == 'Iran (Islamic Republic of)'),'Country_Name']='Iran'
df_countryCode.loc[(df_countryCode.Country_Name == 'Korea (the Republic of)'),'Country_Name']='Korea, South'
df_countryCode.loc[(df_countryCode.Country_Name == "Lao People's Democratic Republic (the)"),'Country_Name']='Laos'
df_countryCode.loc[(df_countryCode.Country_Name == 'Philippines (the)'),'Country_Name']='Philippines'
df_countryCode.loc[(df_countryCode.Country_Name == 'Russian Federation (the)'),'Country_Name']='Russia'
df_countryCode.loc[(df_countryCode.Country_Name == 'Taiwan (Province of China)'),'Country_Name']='Taiwan'
df_countryCode.loc[(df_countryCode.Country_Name == 'Tanzania, United Republic of'),'Country_Name']='Tanzania'
df_countryCode.loc[(df_countryCode.Country_Name == 'United States of America (the)'),'Country_Name']='United States'
df_countryCode.loc[(df_countryCode.Country_Name == 'Venezuela (Bolivarian Republic of)'),'Country_Name']='Venezuela'
df_countryCode.loc[(df_countryCode.Country_Name == 'Viet Nam'),'Country_Name']='Vietnam'

# Checking again for Country names
# List out Countries in df_coffee and not in df_countryCode
df_coffee[~df_coffee.Country_Name.isin(df_countryCode.Country_Name)].Country_Name.unique()

array(['European Union', 'Yemen (Sanaa)'], dtype=object)

In [18]:
# merging df_population with df_countryCode
df_coffee_merged = df_countryCode.merge(df_coffee, on="Country_Name", how="inner")
df_coffee_merged.head()

,Country_Name,Country_Code,Market_Year,Attribute_Description,Value
0,Algeria,DZA,2003,Arabica Production,0.0
1,Algeria,DZA,2003,Bean Exports,0.0
2,Algeria,DZA,2003,Bean Imports,1800.0
3,Algeria,DZA,2003,Domestic Consumption,1805.0
4,Algeria,DZA,2003,Exports,0.0


In [19]:

df_coffee[~df_coffee.Country_Name.isin(df_coffee_merged.Country_Name)].Country_Name.unique()

array(['European Union', 'Yemen (Sanaa)'], dtype=object)

In [20]:
# Concatenating European Union rows into the final dataframe
df_coffee_merged = pd.concat([df_coffee_merged, df_coffee[df_coffee['Country_Name']=="European Union"]])
df_coffee_merged.loc[(df_coffee_merged.Country_Name == 'European Union'),'Country_Code']='EUR'
# Checking again for missing Country names
df_coffee[~df_coffee.Country_Name.isin(df_coffee_merged.Country_Name)].Country_Name.unique()

array(['Yemen (Sanaa)'], dtype=object)

In [21]:
# Checking for null values
df_coffee_merged.isnull().sum()

Country_Name             0
Country_Code             0
Market_Year              0
Attribute_Description    0
Value                    0
dtype: int64

In [22]:
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

In [26]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

class CoffeePCT(Base):
    __tablename__ = 'pct'
    id = Column(Integer, primary_key=True)
    Country_Code = Column(String(3))
    Country_Name = Column(String(30))
    Market_Year = Column(Integer)
    Attribute_Description = Column(String(30))
    Value = Column(Float)

# Create a connection to a SQLite database
engine = create_engine('sqlite:///coffee.sqlite', echo=True)
conn = engine.connect()

sqlite_table = "pct"
df_coffee_merged.to_sql(sqlite_table, conn, if_exists='fail')

# Use this to clear out the db
# Base.metadata.drop_all(engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
Base.metadata.create_all(engine)

2021-02-16 13:28:20,963 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-16 13:28:20,964 INFO sqlalchemy.engine.base.Engine ()
2021-02-16 13:28:21,010 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-16 13:28:21,011 INFO sqlalchemy.engine.base.Engine ()
2021-02-16 13:28:22,348 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("pct")
2021-02-16 13:28:22,349 INFO sqlalchemy.engine.base.Engine ()
2021-02-16 13:28:22,366 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("pct")
2021-02-16 13:28:22,367 INFO sqlalchemy.engine.base.Engine ()
2021-02-16 13:28:22,372 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE pct (
	"index" BIGINT, 
	"Country_Name" TEXT, 
	"Country_Code" TEXT, 
	"Market_Year" BIGINT, 
	"Attribute_Description" TEXT, 
	"Value" FLOAT
)


2021-02-16 13:28:22,373 INFO sqlalchemy.engine.base.Engine ()
2021-02-16 13:28:22,504 INFO sqlalchemy.engine.base.Engi

In [ ]:
# Create a Session Object to Connect to DB
# from sqlalchemy.orm import Session
# session = Session(bind=engine)